In [ ]:

from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
from keras.models import Model
from keras.layers import Input, Dense, Flatten
from keras.applications.vgg16 import VGG16
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf


In [ ]:
IMAGE_SIZE = [224, 224]
train_path = '/content/gdrive/MyDrive/pneumonia/train'
valid_path = '/content/gdrive/MyDrive/pneumonia/test'


In [ ]:
vgg = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

58892288/58889256 [==============================] - 0s 0us/step


In [ ]:
for layer in vgg.layers:
    layer.trainable = False

In [ ]:
x = Flatten()(vgg.output)
prediction = Dense(1, activation='sigmoid')(x)

# create a model object
model = Model(inputs=vgg.input, outputs=prediction)

model.summary()


Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [ ]:
model.compile(
  loss='binary_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [ ]:

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [ ]:
training_set = train_datagen.flow_from_directory(train_path,
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'binary')


Found 5216 images belonging to 2 classes.


In [ ]:
test_set = test_datagen.flow_from_directory(valid_path,
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'binary')

Found 624 images belonging to 2 classes.


In [ ]:

checkpoint = tf.keras.callbacks.ModelCheckpoint('Pneumonia.h5',verbose=1,save_best_only=True)
callbacks =[
            tf.keras.callbacks.EarlyStopping(patience=1,monitor='val_loss'),
            checkpoint
]

In [ ]:
history = model.fit(
  training_set,
  validation_data=test_set,
  epochs=3,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set),
  callbacks=callbacks
)

Epoch 1/3
163/163 [==============================] - 1454s 9s/step - loss: 0.2279 - accuracy: 0.9102 - val_loss: 0.3656 - val_accuracy: 0.8670

Epoch 00001: val_loss improved from inf to 0.36557, saving model to Pneumonia.h5
Epoch 2/3
163/163 [==============================] - 108s 659ms/step - loss: 0.1148 - accuracy: 0.9554 - val_loss: 0.3017 - val_accuracy: 0.9022

Epoch 00002: val_loss improved from 0.36557 to 0.30171, saving model to Pneumonia.h5
Epoch 3/3
163/163 [==============================] - 108s 660ms/step - loss: 0.0901 - accuracy: 0.9672 - val_loss: 0.2463 - val_accuracy: 0.9263

Epoch 00003: val_loss improved from 0.30171 to 0.24626, saving model to Pneumonia.h5


In [ ]:
import cv2
# CATEGORIES = ["Normal", "Infected"]  # will use this to convert prediction num to string value
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
model = tf.keras.models.load_model("/content/Pneumonia.h5")


# def prepare(filepath):
#     # IMG_SIZE = 70  # 50 in txt-based
#     img_array = cv2.imread(filepath)  # read in the image, convert to grayscale
#     new_array = cv2.resize(img_array, (IMAGE_SIZE, IMAGE_SIZE))  # resize image to match model's expected sizing
#     return new_array.reshape(-1, IMAGE_SIZE, IMAGE_SIZE, 1)  # return the image with shaping that TF wants.

In [ ]:
img = image.load_img('/content/gdrive/MyDrive/pneumonia/val/PNEUMONIA/person1954_bacteria_4886.jpeg',target_size=(224,224))
x=image.img_to_array(img)
x=np.expand_dims(x,axis=0)
img_data=preprocess_input(x)
classes=model.predict(img_data)

In [ ]:
# if classes[0][0] > classes[0][1]:
#   print('Normal')
# else:
#   print('Infected')
int(classes)

0

In [ ]:
cp  /content/Pneumonia.h5  /content/gdrive/MyDrive/pneumonia 

In [ ]:
import cv2
# CATEGORIES = ["Normal", "Infected"]  # will use this to convert prediction num to string value

model = tf.keras.models.load_model("/content/Pneumonia.h5")


def prepare(filepath):
    # IMG_SIZE = 70  # 50 in txt-based
    img_array = cv2.imread(filepath, cv2.IMREAD_GRAYSCALE)  # read in the image, convert to grayscale
    new_array = cv2.resize(img_array, (224, 224))  # resize image to match model's expected sizing
    return new_array.reshape(-1, IMAGE_SIZE, IMAGE_SIZE, 1)  # return the image with shaping that TF wants.

In [ ]:
# prediction = model.predict([prepare('/content/gdrive/MyDrive/pneumonia/val/PNEUMONIA/person1950_bacteria_4881.jpeg')])
import tensorflow as tf
import keras
print(tf.__version__)
print(keras.__version__)

2.4.1
2.4.3


In [ ]:
cp  /content/Pneumonia.h5 /content/gdrive/MyDrive/pneumonia